In [75]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path

In [76]:
"""
Reading in data...
data and label are the two variables containing necessary information
data: a pandas dataframe object with rows/index = genes, columns = expression levels of all instances
label: a list, indices correspond to the columns in data
"""
adrenalDataDir = Path("unzipped_data/adrenal_gland")
kidneyDataDir = Path("unzipped_data/kidney")
adrenalData = list(adrenalDataDir.glob("*.txt"))
kidneyData = list(kidneyDataDir.glob("*.txt"))
data = pd.DataFrame(columns = ["gene", "exp"])
label = []
idx = 0
for item in adrenalData:
    instance = pd.read_csv(str(item), header = None, sep = "\t")
    instance.columns = ["gene",f"exp{idx}"]
    if idx == 0:
        data = instance.copy()
    else:
        data = data.join(instance.set_index('gene'), on = "gene")
    idx += 1
    label.append("adrenal")

for item in kidneyData:
    instance = pd.read_csv(str(item), header = None, sep = "\t")
    instance.columns = ["gene",f"exp{idx}"]
    if idx == 0:
        data = instance.copy()
    else:
        data = data.join(instance.set_index('gene'), on = "gene")
    idx += 1
    label.append("kidney")

data = data.set_index("gene")
assert not data.isnull().values.any(), "has some nan values"

In [77]:
def DropEmptyGenes(inputData):
    """
    This function drops all of the genes that are all 0s in both adrenal and kidney samples.
    Args:
        inputData (df(60483,283)): full data array with everything
    Returns:
        newInput (df(58233,283)): full data array without 0s for genes in both samples
    """
    #Row is each gene expression
    newInput = inputData.copy()
    meanByRow = inputData.sum(axis = 1)

    for gene in range(60483):
        if meanByRow[gene] == 0:
            newInput = newInput.drop(inputData.index[gene])

    return newInput

In [78]:
#Remove the genes that are 0 for all samples
dataWO0again = DropEmptyGenes(data)
# print(dataWO0again)

In [79]:
def MeanCenterDF(inputData):
    """
    This function mean centers the data.
    Args:
        inputData (df(58233,283)): full data array with everything
    Returns:
        meanCentered (df(58233,283)): mean centered data
    """
    meanCentered = inputData.apply(lambda x: x-x.mean(), axis=1)

    return meanCentered

In [80]:
#Mean center the data
meancenttest = MeanCenterDF(dataWO0again)

# print(meancenttest)

In [81]:
def NiaveBayesSplitData(inputData):
    """
    This function splits the data into test and train, with a 20% hold out of both the adrenal
    and kidney data.
    Args:
        inputData (df(58233,283)): full data array with everything
    Returns:
        adrenalTrain (df(58233,51)): training x values of adrenal
        adrenalTest (df(58233,13)): testing x values of adrenal
        kidneyTrain (df(58233,175)): training x values of kidney
        kidneyTest (df(58233,44)): testing x values of kidney
    """
    #Split data by class, kidney and adrenal
    dataKidneyOnly = inputData.iloc[:,64:]
    dataAdrenalOnly = inputData.iloc[:,:64]

    #Select random indicies from adrenal values, total 64, selecting 13
    adIdx = np.random.randint(0, 64, size=13) 
    #Sort from max to min for dropping correct row later
    adIdx = np.sort(adIdx)
    adIdx = adIdx[::-1]

    #Build adrendal test and train set, initialize with the fist value added to test and
    #dropped from train
    adrenalTest = dataAdrenalOnly.iloc[:,adIdx[0]:adIdx[0]+1]
    adrenalTrain = dataAdrenalOnly.drop(columns=dataAdrenalOnly.columns[adIdx[0]])
    #Go through list of indicies to add and drop from test and train respectively
    for i in range(1,13):
        adrenalTest = pd.concat([adrenalTest, dataAdrenalOnly.iloc[:,adIdx[i]:adIdx[i]+1]], axis=1)
        adrenalTrain = adrenalTrain.drop(columns=adrenalTrain.columns[adIdx[i]])


    #Select random indicies from kidney values, total 219, selecting 44
    kidIdx = np.random.randint(0, 219, size=44)
    #Sort from max to min for dropping correct row later
    kidIdx = np.sort(kidIdx)
    kidIdx = kidIdx[::-1]

    #Build kidney test and train set, initialize with the fist value added to test and
    #dropped from train
    kidneyTest = dataKidneyOnly.iloc[:,kidIdx[0]:kidIdx[0]+1]
    kidneyTrain = dataKidneyOnly.drop(columns=dataKidneyOnly.columns[kidIdx[0]])
    #Go through list of indicies to add and drop from test and train respectively
    for j in range(1,44):
        kidneyTest = pd.concat([kidneyTest, dataKidneyOnly.iloc[:,kidIdx[j]:kidIdx[j]+1]], axis=1)
        kidneyTrain = kidneyTrain.drop(columns=kidneyTrain.columns[kidIdx[j]])


    return adrenalTrain, adrenalTest, kidneyTrain, kidneyTest

In [82]:
def CalculateMeanAndStdDevForTrain(adrenalTrain, kidneyTrain):
    """
    This function  calculates the mean and standard deviation. 
    Stores them in numpy arrays for further calulations.
    Args:
        adrenalTrain (df(58233,51)): full train df of adrenal
        kidneyTrain (df(58233,175)): full train df of kidney
    Returns:
        meanA (np.array(58233,)): mean of each column of adrenal train
        stdDevA (np.array(58233,)): standard deviation of each column of adrenal train
        meanK (np.array(58233,)): mean of each column of kidney train
        stdDevK (np.array(58233,)): standard deviation of each column of kidney train
    """

    #Calulate mean across the columns
    meanA = adrenalTrain.mean(axis=1)
    meanK = kidneyTrain.mean(axis=1)
    #Convert to numpy
    meanA = meanA.to_numpy()
    meanK = meanK.to_numpy()

    #Calulate standard deviation across the columns
    stdDevA = adrenalTrain.std(axis=1)
    stdDevK = kidneyTrain.std(axis=1)
     #Convert to numpy
    stdDevA = stdDevA.to_numpy()
    stdDevK = stdDevK.to_numpy()

    return meanA, stdDevA, meanK, stdDevK

In [83]:
#Split data into adrenal and kidney test and train
aTrain, aTest, kTrain, kTest = NiaveBayesSplitData(meancenttest)

#Caclulate mean and std dev on train for adrenal and kidney
meanA0, stdDevA0, meanK0, stdDevK0 = CalculateMeanAndStdDevForTrain(aTrain, kTrain)

In [20]:
# dataWithout0 = DropEmptyGenes(data) 
# meanCentData = MeanCenterDF(dataWithout0) 
# aTrain, aTest, kTrain, Ktest = NiaveBayesSplitData(meancenttest)
# meanA0, stdDevA0, meanK0, stdDevK0 = CalculateMeanAndStdDevForTrain(aTest, Ktest)
# print(np.where(stdDevA0  == 0))
# print(np.where(stdDevK0 == 0))

In [11]:
#take out all rows with 0 that both -
#mean center -
# split data by training and test -
#Split data by class, kidney_train and adrenal_train, test (both, labels hidden) -
# meanA0, stdDevA0, meanK0, stdDevK = CalculateMenaAndStdDevForTrain(adrenal_train,kidney_train) -
# remove rows that have 0 std dev in either stddevA0 or stdDevK from kidney_train,adrenal_train, and test -
# calculateProb on test, append to a initially empty list
# check that no values in the list is nan; sum(np.isnan(np.array(initially empty list)))=0
# if not, you're all set
# if so, let me know
# dataKidneyOnlyMean = meancenttest.iloc[:,64:]
# dataAdrenalOnlyMean = meancenttest.iloc[:,:64]
# meanA0, stdDevA0, meanK0, stdDevK0 = CalculateMeanAndStdDevForTrain(dataAdrenalOnlyMean, dataKidneyOnlyMean)

In [33]:
# print(meanCentData.iloc[58195])
sdA0s = np.where(stdDevA0  == 0)
sdk0s = np.where(stdDevK0 == 0)
print("lenth os sda is", len(sdA0s), sdA0s, "length of sdk is", len(sdk0s), sdk0s)
both0s = np.concatenate((sdA0s, sdk0s), axis=None)
print(sdk0s, sdA0s)
print("both is", both0s, "len of both is", len(both0s))
# bothSort = np.sort(both0s)
# print(bothSort, "len of sort is", len(bothSort))
bothUnique = np.unique(both0s)
print("both unique is", bothUnique, len(bothUnique))
# print("len of kid is", sdk0s.size, "len of a is", len(sdA0s), "len of both is", len(both0s))

lenth os sda is 1 (array([ 1785, 12929, 13116, 13304, 14336, 14850, 16363, 16961, 17673,
       18147, 18425, 18432, 18451, 18472, 18619, 18775, 18800, 18813,
       19146, 19266, 20406, 20446, 20465, 20541, 20825, 21024, 21129,
       21169, 21208, 21250, 21270, 21673, 21727, 21743, 21788, 21836,
       21880, 21946, 22801, 23072, 23301, 23757, 23822, 23874, 23881,
       23926, 23957, 23958, 23966, 23982, 24062, 24080, 24103, 24112,
       24193, 24213, 24274, 24311, 24495, 24521, 24738, 24864, 25111,
       25632, 25703, 25760, 26127, 26354, 26404, 26722, 28783, 29211,
       29300, 29708, 30022, 30383, 30621, 31662, 32781, 32962, 33243,
       33700, 34011, 34600, 34702, 35281, 35377, 36808, 37362, 37663,
       37673, 38282, 38715, 38742, 38758, 39123, 39237, 39415, 40349,
       40358, 40416, 40419, 40422, 40516, 40534, 40571, 40595, 40647,
       40665, 40707, 40752, 40865, 41214, 41517, 42120, 42399, 42526,
       42818, 43244, 43529, 44714, 44938, 46763, 47899, 47972, 48026,
 

In [15]:
# np.sum(data.loc[dataKidneyOnlyMean.index[np.where(stdDevA0  == 0)],:].to_numpy().mean(axis = 1) == 0)

0

In [84]:
def RemoveStdDev0sFromData(standardDevA, standardDevK):
    """
    This function tracks all the genes that result in 0 in either kidney or adrenal.
    Merges the list, only keeps the unique values and then stores to be deleted.
    Args:
        standardDevA (np.array(58233,)): standard deviation of each column of adrenal train
        standardDevK (np.array(58233,)): standard deviation of each column of kidney train
    Returns:
        bothUnique (np.array(random,)): list of indices to be remove, since the random split
        on test and train, the indices may be different on different runs.
    """
    #Where are the 0s?
    sdA0s = np.array(np.where(standardDevA  == 0))
    sdk0s = np.array(np.where(standardDevK == 0))
    #Merge lists
    both0s = np.append(sdA0s, sdk0s)
    #Sort lists
    bothSort = np.sort(both0s)
    #Keep unique values only
    bothUnique = np.unique(bothSort)

    #returns list of indicies to be eliminated
    return bothUnique

In [85]:
def DropEmptyStandardDevs(inputData, uniqueListOf0):
    """
    This function takes the list of indices to be deleted and deletes them from the data.
    Args:
        inputData (np.array(58233,283)): full data set
        uniqueListOf0 (np.array(random,)): list of indices to be deleted
    Returns:
        newInput (np.array(58233-random,)): data with the indices removed
    """
    #Row is each gene expression
    newInput = inputData.copy()

    for index in range(len(uniqueListOf0)):
            newInput = newInput.drop(inputData.index[uniqueListOf0[index]])

    return newInput

In [86]:
#Check which genes give standard deviation of 0
listToBeDeleted = RemoveStdDev0sFromData(stdDevA0, stdDevK0)

#Remove those genes from test and train
print(kTrain.shape, aTrain.shape, kTest.shape, aTest.shape)
kTrain = DropEmptyStandardDevs(kTrain, listToBeDeleted)
aTrain = DropEmptyStandardDevs(aTrain, listToBeDeleted)
kTest = DropEmptyStandardDevs(kTest, listToBeDeleted)
aTest = DropEmptyStandardDevs(aTest, listToBeDeleted)
print(kTrain.shape, aTrain.shape, kTest.shape, aTest.shape)

(58233, 175) (58233, 51) (58233, 44) (58233, 13)
(57982, 175) (57982, 51) (57982, 44) (57982, 13)


In [87]:
#Remove those genes from all means and standard deviations
print(meanA0.shape, stdDevA0.shape, meanK0.shape, stdDevK0.shape)
meanA0 = np.delete(meanA0, listToBeDeleted)
stdDevA0 = np.delete(stdDevA0, listToBeDeleted)
meanK0 = np.delete(meanK0, listToBeDeleted)
stdDevK0 = np.delete(stdDevK0, listToBeDeleted)
print(meanA0.shape, stdDevA0.shape, meanK0.shape, stdDevK0.shape)

(58233,) (58233,) (58233,) (58233,)
(57982,) (57982,) (57982,) (57982,)


In [189]:
# meanA0, stdDevA0, meanK0, stdDevK0 = CalculateMeanAndStdDevForTrain(aTest, Ktest)
# print(meanA0.shape, stdDevA0.shape)
# print(meanK0.shape, stdDevK0.shape)
# print(meanA0, stdDevA0)

In [88]:
def CalculateProb(x, mean, stdDev):
    """
    This function calculates the probability for a variable based on a mean and standard
    deviation that is fed.
    Args:
        x (float): input value
        mean (float): mean value
        stdDev (float): standard deviation
    Returns:
        prob (float): the probability
    """
    prob = (1/(stdDev*np.sqrt(2*np.pi)))*np.exp((-1/(2*(stdDev**2)))*(x-mean)**2)
    return prob

In [89]:
def CalculateProbNegLog(x, mean, stdDev):
    """
    This function calculates the negative log probability for a variable based on a mean and standard
    deviation that is fed.
    Args:
        x (float): input value
        mean (float): mean value
        stdDev (float): standard deviation
    Returns:
        prob (float): the probability
    """
    prob = (1/(stdDev*np.sqrt(2*np.pi))) * np.exp((-1/(2*(stdDev**2)))*(x-mean)**2)
    log = -np.log(prob)
    return log

In [90]:
def CalculateProbChange0BeforeLog(x, mean, stdDev):
    """
    This function calculates the negative log probability for a variable based on a mean and standard
    deviation that is fed.
    Args:
        x (float): input value
        mean (float): mean value
        stdDev (float): standard deviation
    Returns:
        prob (float): the probability
    """

    prob = (1/(stdDev*np.sqrt(2*np.pi))) * np.exp((-1/(2*(stdDev**2)))*(x-mean)**2)
    if prob == 0:
        prob = 1e-10 #Epsilon value

    log = -np.log(prob)
    return log

In [91]:
def CalculateProbBasedOnClass(aTest, kTest, meanA, stdDevA, meanK, stdDevK):
    """
    This function calculates the probability based on kidney and adrenal class. It uses the 
    -log instead of the regular probability to get past the values vanishing after being
    multiplied 50k times (gene length).
    Args:
        aTest (df(58233-?,13)): full test of adrenal
        kTest (df(58233-?,44)): full test of kidney
        meanA (np.array(58233-?,)): mean of each column of adrenal train
        stdDevA (np.array(58233-?,)): standard deviation of each column of adrenal train
        meanK (np.array(58233-?,)): mean of each column of kidney train
        stdDevK (np.array(58233-?,)): standard deviation of each column of kidney train
    Returns:
        xClassProb (np.array(57,2)): matrix with probability of adrenal and kidney,
        col 0 = adrenal col 1 = kidney. Row 0:13 = adrenal and row 13:57 = kidney
        correspond to true classes.
    """

    #Initialize array that will hold class probabilities of adrenal(0) and kidney(1) for each
    #x input value
    xClassProb = np.zeros((57, 2))

    #Calcualte P(x|class) for each gene in each class for each sample
    #Go through every gene for all adrenal samples
    for samplesA in range(13):
        for gene in range(len(aTest)):
            if gene == 0:
                xClassProb[samplesA,0] = CalculateProbChange0BeforeLog(aTest.iloc[gene,samplesA], meanA[gene], stdDevA[gene])
                xClassProb[samplesA,1] = CalculateProbChange0BeforeLog(aTest.iloc[gene,samplesA], meanK[gene], stdDevK[gene])
            else:
                #Calculate probability of these samples from adrenal test as being classified as adrenal
                xClassProb[samplesA,0] += CalculateProbChange0BeforeLog(aTest.iloc[gene,samplesA], meanA[gene], stdDevA[gene])

                #Calculate probability of these samples from adrenal test as being classified as kidney
                xClassProb[samplesA,1] += CalculateProbChange0BeforeLog(aTest.iloc[gene,samplesA], meanK[gene], stdDevK[gene])
    
    #Go throgh every gene for all kindey samples
    for samplesK in range(44):
        for gene in range(len(kTest)):
            if gene == 0:
                xClassProb[13+samplesK,0] = CalculateProbChange0BeforeLog(kTest.iloc[gene,samplesK], meanA[gene], stdDevA[gene])
                xClassProb[13+samplesK,1] = CalculateProbChange0BeforeLog(kTest.iloc[gene,samplesK], meanK[gene], stdDevK[gene])
            else:
                #Starting at positiong [14,0] in [57,2] matrix
                #Calculate probability of these samples from kidney test as being classified as adrenal
                xClassProb[13+samplesK,0] += CalculateProbChange0BeforeLog(kTest.iloc[gene,samplesK], meanA[gene], stdDevA[gene])
                #Calculate probability of these samples from kidney test as being classified as kidney
                xClassProb[13+samplesK,1] += CalculateProbChange0BeforeLog(kTest.iloc[gene,samplesK], meanK[gene], stdDevK[gene])

    #After done calculating P(x|class) must now calculate P(x|class)*P(class)
    probAdrendal = -np.log(64/283)
    probKidney = -np.log(219/283)
    #Multiple the top 13 rows of adrenal sample 
    xClassProb[:,:1] = xClassProb[:,:1] + probAdrendal
    xClassProb[:,1:] = xClassProb[:,1:] + probKidney
    # print("classprob after multiplication", xClassProb)

    return xClassProb


In [97]:
#Calculate the probabilities for each sample for both kidney and adrenal
classProb = CalculateProbBasedOnClass(aTest, kTest, meanA0, stdDevA0, meanK0, stdDevK0)

In [94]:
def WhichProbIsGreater(inputData):
    """
    This function measures which probability is greater. For a log value that would be the
    most negative, for a negative log value that is the smallest positive.
    Args:
        xClassProb (np.array(57,2)): matrix with probability of adrenal and kidney,
        col 0 = adrenal col 1 = kidney. Row 0:13 = adrenal and row 13:57 = kidney
        correspond to true classes.
    Returns:
        finalProb (np.array(57,)): list of final probabilities predicted
    """
    finalProb = np.zeros((57, 1))
    for index in range(57):
        #If probability is in favor of adrenal, set value to 0
        if inputData[index, 0] < inputData[index, 1]:
            finalProb[index] = 0
        #If probability is in favor of kidney, set value to 1
        else:
            finalProb[index] = 1
    return finalProb

In [99]:
#Feed the result through this function to get the most probable
finalProb= WhichProbIsGreater(classProb)
print(finalProb)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]]


In [100]:
def Precision(inputData):
    """
    This function measures precision.
    Args:
        inputData (np.array(57,)): list of final probabilities predicted
    Returns:
       precisionVal (int): precision
    """
    #44 true values of kidney assigned as 1
    truePos = 44
    #Select the adrenal test set
    adrenalTrue = inputData[:13]
    #Add all values together since it would be a false positive since adrenal is 0
    falsePos = np.sum(adrenalTrue)
    #Calculate precision
    precisionVal = truePos / (truePos+falsePos)
    return precisionVal

In [101]:
#Calculate the precision
precision= Precision(finalProb)
print(precision)

1.0


In [102]:
def Recall(inputData):
    """
    This function measures recall.
    Args:
        inputData (np.array(57,)): list of final probabilities predicted
    Returns:
       recallVal (int): recall
    """
    #44 true values of kidney assigned as 1
    truePos = 44
    #Select the kidney test set
    kidneyTrue = inputData[13:]
    #False negative are kidney test that are 0 and should be 1
    falseNeg = len(kidneyTrue) - np.sum(kidneyTrue)
    #Calculate recall
    recallVal = truePos / (truePos + falseNeg)
    return recallVal


In [103]:
#Calculate the recall
recall= Recall(finalProb)
print(recall)

0.9166666666666666


In [104]:
def F1Score(inputData):
    """
    This function measure F1 score.
    Args:
        inputData (np.array(57,)): list of final probabilities predicted
    Returns:
       f1 (int): f1 score
    """
    #Calculate precision
    precisionVar = Precision(inputData)
    #Calculate recall
    recallVar = Recall(inputData)
    #Calculate F1 score
    f1 = 2*((precisionVar*recallVar)/(precisionVar+recallVar))
    return f1

In [105]:
#Calculate the f1 score
f1 = F1Score(finalProb)
print(f1)

0.9565217391304348
